In [0]:
!pip install --upgrade pip
!pip install tensorflow-gpu

     |████████████████████████████████| 1.4MB 7.7MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 421.8 MB 3.8 kB/s 
     |████████████████████████████████| 448 kB 85.1 MB/s 
     |████████████████████████████████| 3.8 MB 77.7 MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.1.1 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 1.15.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling ten

In [0]:
# SET UP

# Data loader
import tensorflow as tf
print(tf.__version__)
from glob import glob
import numpy as np
import random
import scipy
import os
import cv2 as cv
from tensorflow import keras

# Network
#from keras.layers import Input, Conv2D, Conv2DTranspose, Concatenate
from tensorflow.keras.applications.vgg19 import VGG19
#from keras.models import Model

from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, UpSampling2D, Conv2DTranspose, Reshape, Dropout, concatenate, Concatenate, multiply, add, MaxPooling2D, Lambda, Activation, subtract, Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
#from keras_contrib.layers.normalization import InstanceNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import imageio
from tensorflow.python.keras import backend as K
from tensorflow.keras.models import Model
#from data_load import Dataloader
from scipy import misc
#from glob import glob
#import tensorflow as tf
import numpy as np
import scipy
import platform
#import keras
import os
import random
#import Network
#import utls

# Access Drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
drive.mount('/content/drive')

2.1.0


Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#filep = '/content/drive/My Drive/726FinalProj/dataset/train_test/*.png'
# ==================================================================
# PARAMS
ROOT_DIR = '/content/drive/My Drive/726FinalProj'
DATA_FOLDER_HI = '/high' #'/high_small_size'
DATA_FOLDER_LO = '/low'  #'/low_small_size'
BATCH_SIZE = 8 # original: 16
NUM_EPOCHS = 1  # original: 5
STEPS_PER_EPOCH = 1  # num_samples // batch_size, in general, for our data should be 30.
img_rows = 256
img_cols = 256
VAL_DIR = '/val_images2'

BASE_LEARN_RATE    = 1*1e-03

LOAD_FROM_WEIGHTS  = True
MODEL_WEIGHTS_FILE = '12em.h5'
RESUMED_LEARN_RATE = 0.0 #3.772367e-11 #1.5039908e-06 #7.5471934e-11
# ==================================================================

In [0]:

class Dataloader():
    def __init__(self, dataset_name, crop_shape=(256, 256)):
        self.dataset_name = dataset_name
        self.crop_shape = crop_shape

    def imread_color(self, path):
        img = cv.imread(path, cv.IMREAD_COLOR | cv.IMREAD_ANYDEPTH) /255.
        b, g, r = cv.split(img)
        img_rgb = cv.merge([r, g, b])
        return img_rgb

    def imwrite(self, path, img):
        r, g, b = cv.split(img)
        img_rgb = cv.merge([b, g, r])
        cv.imwrite(path, img_rgb)

    def load_data(self, batch_size=16):
        path = glob(ROOT_DIR + '/dataset/our485' + DATA_FOLDER_HI + '/*.png')
        self.n_batches = int(len(path) / batch_size)
        while 1:
            random.shuffle(path)
            for i in range(self.n_batches - 1):
                batch_path = path[i * batch_size:(i + 1) * batch_size]
                input_imgs = np.empty((batch_size, self.crop_shape[0], self.crop_shape[1], 6), dtype="float32")
                gt = np.empty((batch_size, self.crop_shape[0], self.crop_shape[1], 3), dtype="float32")

                number = 0
                for img_B_path in batch_path:
                    img_B = self.imread_color(img_B_path)

                    path_split = os.path.split(img_B_path)
                    path_prefix = os.path.split(path_split[0])
                    path_A = path_prefix[0] + DATA_FOLDER_LO + '/' + path_split[1]

                    # path_mid = os.path.split(img_B_path)
                    # path_A_1 = path_mid[0] + '_' + self.dataset_name
                    # path_A = os.path.join(path_A_1, path_mid[1])
                    
                    img_A = self.imread_color(path_A)

                    nw = random.randint(0, img_B.shape[0] - self.crop_shape[0])
                    nh = random.randint(0, img_B.shape[1] - self.crop_shape[1])

                    crop_img_A = img_A[nw:nw + self.crop_shape[0], nh:nh + self.crop_shape[1], :]
                    crop_img_B = img_B[nw:nw + self.crop_shape[0], nh:nh + self.crop_shape[1],:]
                    
                    # im = Image.fromarray(crop_img_A)
                    # im.show()
                    # im = Image.fromarray(crop_img_B)
                    # im.show()

                    if np.random.randint(2, size=1)[0] == 1:  # random flip
                        crop_img_A = np.flipud(crop_img_A)
                        crop_img_B = np.flipud(crop_img_B)
                    if np.random.randint(2, size=1)[0] == 1:
                        crop_img_A = np.fliplr(crop_img_A)
                        crop_img_B = np.fliplr(crop_img_B)
                    if np.random.randint(2, size=1)[0] == 1:  # random transpose
                        crop_img_A = np.transpose(crop_img_A, (1, 0, 2))
                        crop_img_B = np.transpose(crop_img_B, (1, 0, 2))

                    input_imgs[number, :, :, :] = np.concatenate([crop_img_A, crop_img_B], axis=-1)
                    gt[number, :, :, :] = crop_img_B
                    number += 1
                yield input_imgs, gt

def build_vgg():
    vgg_model = VGG19(include_top=False, weights='imagenet')
    vgg_model.trainable = False
    return Model(inputs=vgg_model.input, outputs=vgg_model.get_layer('block3_conv4').output)

def build_mbllen(input_shape):

    def EM(input, kernal_size, channel):
        conv_1 = Conv2D(channel, (3, 3), activation='relu', padding='same', data_format='channels_last')(input)
        conv_2 = Conv2D(channel, (kernal_size, kernal_size), activation='relu', padding='valid', data_format='channels_last')(conv_1)
        conv_3 = Conv2D(channel*2, (kernal_size, kernal_size), activation='relu', padding='valid', data_format='channels_last')(conv_2)
        conv_4 = Conv2D(channel*4, (kernal_size, kernal_size), activation='relu', padding='valid', data_format='channels_last')(conv_3)
        conv_5 = Conv2DTranspose(channel*2, (kernal_size, kernal_size), activation='relu', padding='valid', data_format='channels_last')(conv_4)
        conv_6 = Conv2DTranspose(channel, (kernal_size, kernal_size), activation='relu', padding='valid', data_format='channels_last')(conv_5)
        res = Conv2DTranspose(3, (kernal_size, kernal_size), activation='relu', padding='valid', data_format='channels_last')(conv_6)
        return res

    inputs = Input(shape=input_shape)
    FEM = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_last')(inputs)
    EM_com = EM(FEM, 5, 8)

    for j in range(4):                                                                                                                                      # MODIFICATION: WAS 3!
        for i in range(0, 3):
            FEM = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_last')(FEM)
            EM1 = EM(FEM, 5, 8)
            EM_com = Concatenate(axis=3)([EM_com, EM1])

    outputs = Conv2D(3, (1, 1), activation='relu', padding='same', data_format='channels_last')(EM_com)
    return Model(inputs, outputs)

def bright_mae(y_true, y_pred):
    return K.mean(K.abs(y_pred[:,:,:,:3] - y_true[:,:,:,:3]))

def bright_mse(y_true, y_pred):
    return K.mean((y_pred[:,:,:,:3] - y_true[:,:,:,:3])**2)

def bright_AB(y_true, y_pred):
            return K.abs(K.mean(y_true[:,:,:,:3])-K.mean(y_pred[:,:,:,:3]))

def log10(x):
    numerator = K.log(x)
    denominator = K.log(K.constant(10, dtype=numerator.dtype))
    return numerator / denominator

def bright_psnr(y_true, y_pred):
    mse = K.mean((K.abs(y_pred[:,:,:,:3] - y_true[:,:,:,:3])) ** 2)
    max_num = 1.0
    psnr = 10 * log10(max_num ** 2 / mse)
    return psnr

def _tf_fspecial_gauss(size, sigma):
    """Function to mimic the 'fspecial' gaussian MATLAB function
    """
    x_data, y_data = np.mgrid[-size//2 + 1:size//2 + 1, -size//2 + 1:size//2 + 1]

    x_data = np.expand_dims(x_data, axis=-1)
    x_data = np.expand_dims(x_data, axis=-1)

    y_data = np.expand_dims(y_data, axis=-1)
    y_data = np.expand_dims(y_data, axis=-1)

    x = tf.constant(x_data, dtype=tf.float32)
    y = tf.constant(y_data, dtype=tf.float32)

    g = tf.exp(-((x**2 + y**2)/(2.0*sigma**2)))
    return g / tf.reduce_sum(g)

def tf_ssim(img1, img2, cs_map=False, mean_metric=True, size=11, sigma=1.5):
    window = _tf_fspecial_gauss(size, sigma) # window shape [size, size]
    K1 = 0.01
    K2 = 0.03
    L = 1  # depth of image (255 in case the image has a differnt scale)
    C1 = (K1*L)**2
    C2 = (K2*L)**2
    mu1 = tf.nn.conv2d(img1, window, strides=[1,1,1,1], padding='VALID')
    mu2 = tf.nn.conv2d(img2, window, strides=[1,1,1,1],padding='VALID')
    mu1_sq = mu1*mu1
    mu2_sq = mu2*mu2
    mu1_mu2 = mu1*mu2
    sigma1_sq = tf.nn.conv2d(img1*img1, window, strides=[1,1,1,1],padding='VALID') - mu1_sq
    sigma2_sq = tf.nn.conv2d(img2*img2, window, strides=[1,1,1,1],padding='VALID') - mu2_sq
    sigma12 = tf.nn.conv2d(img1*img2, window, strides=[1,1,1,1],padding='VALID') - mu1_mu2
    if cs_map:
        value = (((2*mu1_mu2 + C1)*(2*sigma12 + C2))/((mu1_sq + mu2_sq + C1)*
                            (sigma1_sq + sigma2_sq + C2)),
                        (2.0*sigma12 + C2)/(sigma1_sq + sigma2_sq + C2))
    else:
        value = ((2*mu1_mu2 + C1)*(2*sigma12 + C2))/((mu1_sq + mu2_sq + C1)*
                            (sigma1_sq + sigma2_sq + C2))

    if mean_metric:
        value = tf.reduce_mean(value)
    return value

def tf_ms_ssim(img1, img2, mean_metric=True, level=5):
    weight = tf.constant([0.0448, 0.2856, 0.3001, 0.2363, 0.1333], dtype=tf.float32)
    mssim = []
    mcs = []
    for l in range(level):
        ssim_map, cs_map = tf_ssim(img1, img2, cs_map=True, mean_metric=False)
        mssim.append(tf.reduce_mean(ssim_map))
        mcs.append(tf.reduce_mean(cs_map))
        filtered_im1 = tf.nn.avg_pool(img1, [1,2,2,1], [1,2,2,1], padding='SAME')
        filtered_im2 = tf.nn.avg_pool(img2, [1,2,2,1], [1,2,2,1], padding='SAME')
        img1 = filtered_im1
        img2 = filtered_im2

    # list to tensor of dim D+1
    mssim = tf.stack(mssim, axis=0)
    mcs = tf.stack(mcs, axis=0)

    value = (tf.reduce_prod(mcs[0:level-1]**weight[0:level-1])*
                                    (mssim[level-1]**weight[level-1]))

    if mean_metric:
        value = tf.reduce_mean(value)
    return value

def bright_SSIM(y_true, y_pred):
    SSIM_loss = tf_ssim(tf.expand_dims(y_pred[:,:,:,0], -1), tf.expand_dims(y_true[:,:,:,0], -1))+tf_ssim(tf.expand_dims(y_pred[:,:,:,1], -1), tf.expand_dims(y_true[:,:,:,1], -1)) + tf_ssim(tf.expand_dims(y_pred[:,:,:,2], -1), tf.expand_dims(y_true[:,:,:,2], -1))
    return SSIM_loss/3

def psnr_cau(y_true, y_pred):
    mse = np.mean((np.abs(y_pred - y_true)) ** 2)
    max_num = 1.0
    psnr = 10 * np.log10(max_num ** 2 / mse)
    return psnr

def save_model(model, name, epoch, batch_i):
    modelname = './Res_models/' + str(epoch) + '_' + str(batch_i) + name + '.h5'
    model.save_weights(modelname)

def imread_color(path):
    img = cv.imread(path, cv.IMREAD_COLOR | cv.IMREAD_ANYDEPTH) / 255.
    b, g, r = cv.split(img)
    img_rgb = cv.merge([r, g, b])
    return img_rgb
    # return scipy.misc.imread(path, mode='RGB').astype(np.float) / 255.

def imwrite(path, img):
    r, g, b = cv.split(img*255)
    img_rgb = cv.merge([b, g, r])
    cv.imwrite(path, img_rgb)
    # scipy.misc.toimage(img * 255, high=255, low=0, cmin=0, cmax=255).save(path)

def range_scale(x):
    return x * 2 - 1.


In [0]:
def my_loss(y_true, y_pred):
    MAE_loss = K.mean(K.abs(y_pred[:,:,:,:3] - y_true))
    SSIM_loss = tf_ssim(tf.expand_dims(y_pred[:, :, :, 0], -1),tf.expand_dims(y_true[:, :, :, 0], -1)) + tf_ssim(
        tf.expand_dims(y_pred[:, :, :, 1], -1), tf.expand_dims(y_true[:, :, :, 1], -1)) + tf_ssim(
        tf.expand_dims(y_pred[:, :, :, 2], -1), tf.expand_dims(y_true[:, :, :, 2], -1))
    VGG_loss = K.mean(K.abs(y_pred[:, :, :, 3:19] - y_pred[:, :, :, 19:35]))

    percent = 0.4
    index = int(256 * 256 * percent - 1)
    gray1 = 0.39 * y_pred[:, :, :, 0] + 0.5 * y_pred[:, :, :, 1] + 0.11 * y_pred[:, :, :, 2]
    gray = tf.reshape(gray1, [-1, 256 * 256])
    gray_sort = tf.nn.top_k(-gray, 256 * 256)[0]
    yu = gray_sort[:, index]
    yu = tf.expand_dims(tf.expand_dims(yu, -1), -1)
    mask = tf.dtypes.cast((gray1 <= yu), tf.float32)
    mask1 = tf.expand_dims(mask, -1)
    mask = tf.concat([mask1, mask1, mask1], -1)

    low_fake_clean = tf.multiply(mask, y_pred[:, :, :, :3])
    high_fake_clean = tf.multiply(1 - mask, y_pred[:, :, :, :3])
    low_clean = tf.multiply(mask, y_true[:, :, :, :])
    high_clean = tf.multiply(1 - mask, y_true[:, :, :, :])
    Region_loss = K.mean(K.abs(low_fake_clean - low_clean) * 4 + K.abs(high_fake_clean - high_clean))

    loss = MAE_loss + VGG_loss/3. + 3 - SSIM_loss + Region_loss
    return loss

if not os.path.isdir(ROOT_DIR + '/val_images2'):
    os.makedirs(ROOT_DIR + '/val_images2')
if not os.path.isdir(ROOT_DIR + '/logs'):
    os.makedirs(ROOT_DIR + '/logs')
if not os.path.isdir(ROOT_DIR + '/models'):
    os.makedirs(ROOT_DIR + '/models')

def f1(x):
    return x[:, :, :, :3]

def f2(x):
    return x[:, :, :, 3:]

def f3(x):
    return tf.reshape(x,[-1, 256, 256, 16])

img_channels = 3
crop_shape = (img_rows, img_cols, img_channels)
input_shape = (img_rows, img_cols, img_channels*2)
dataset_name = 'lol'
data_loader = Dataloader(dataset_name=dataset_name, crop_shape=(img_rows, img_cols))

# ==================================================================
# Build the network
mbllen = build_mbllen(crop_shape)
# mbllen.load_weights('./1_dark2_color_identity_param.h5')

Input_MBLLEN = Input(shape=input_shape)
img_A = Lambda(f1)(Input_MBLLEN)
img_B = Lambda(f2)(Input_MBLLEN)

# VGG19 feature, content loss
vgg = build_vgg()
vgg.trainable = False

fake_B = mbllen(img_A)
vgg_fake = Lambda(range_scale)(fake_B)
fake_features = vgg(vgg_fake)
fake_features = Lambda(f3)(fake_features)

img_B_vgg = Lambda(range_scale)(img_B)
imgb_features = vgg(img_B_vgg)
imgb_features = Lambda(f3)(imgb_features)

output_com = concatenate([fake_B, fake_features, imgb_features], axis=3)

if LOAD_FROM_WEIGHTS == True:
  start_learn_rate = RESUMED_LEARN_RATE
else:
  start_learn_rate = BASE_LEARN_RATE

opt = Adam(lr=start_learn_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
combined = Model(inputs=Input_MBLLEN, outputs=output_com)
combined.compile(loss=my_loss,
                 metrics=[bright_mae, bright_mse, bright_psnr, bright_SSIM, bright_AB],
                 optimizer=opt)

# plot_model(mbllen, to_file='./model.png', show_shapes=True)
combined.summary()

# ==================================================================
#
def scheduler(epoch):
    lr = K.eval(combined.optimizer.lr)
    print("LR =", lr)
    lr = lr * 0.99
    return lr

# we will use this sequence for generating the random patch positions so that they are the same for each epoch
# patch_positions = []
num_epoch = 0
class Show_History(keras.callbacks.Callback):
    def on_epoch_end(self, val_loss=None, logs=None):
        # save model
        global num_epoch
        global dataset_name
        global img_rows
        global img_cols
        global mbllen
        num_epoch += 1
        modelname = ROOT_DIR + '/models/' + dataset_name + '/' + 'only_lol/' + str(num_epoch) + '_' + dataset_name + '_base.h5'           # MODIFICATION : added 'layers_2/'
        mbllen.save_weights(modelname)

        # test val data
        path = glob(ROOT_DIR + '/dataset/eval15/high/*.png')
        number = 0
        psnr_ave = 0


        for i in range(len(path)):
            if i>15:
                break

            img_B_path = path[i]
            img_B = imread_color(img_B_path)

            # path_mid = os.path.split(img_B_path)
            # path_A_1 = path_mid[0] + '_' + dataset_name
            # path_A = os.path.join(path_A_1, path_mid[1])

            path_split = os.path.split(img_B_path)
            path_prefix = os.path.split(path_split[0])
            path_A = path_prefix[0] + '/low/' + path_split[1]

            img_A = imread_color(path_A)

            nw = 100 #random.randint(0, img_A.shape[0] - img_rows)
            nh = 150 #random.randint(0, img_A.shape[1] - img_cols)

            crop_img_A = img_A[nw:nw + img_rows, nh:nh + img_cols, :]
            crop_img_B = img_B[nw:nw + img_rows, nh:nh + img_cols, :]

            crop_img_A = crop_img_A[np.newaxis, :]
            crop_img_B = crop_img_B[np.newaxis, :]

            fake_B = mbllen.predict(crop_img_A)
            identy_B = mbllen.predict(crop_img_B)

            out_img = np.concatenate([crop_img_A, fake_B, crop_img_B, identy_B], axis=2)
            out_img = out_img[0, :, :, :]

            fake_B = fake_B[0, :, :, :]
            img_B = crop_img_B[0, :, :, :]

            clean_psnr = psnr_cau(fake_B, img_B)
            L_psnr = ("%.4f" % clean_psnr)

            number += 1
            psnr_ave += clean_psnr

            im_name = path_split[1].split('.')[0]
            filename = os.path.basename(path[i])
            img_name = ROOT_DIR + VAL_DIR + '/' + str(num_epoch) + '_' + im_name + '_' + L_psnr + '_' + filename
            imwrite(img_name, out_img)
        psnr_ave /= number
        print('------------------------------------------------')
        print("[Epoch %d]  [PSNR_AVE :%f]" % (num_epoch,  psnr_ave))
        print('------------------------------------------------')

    def on_batch_end(self, batch, logs={}):
        print(' - LR = ', K.eval(self.model.optimizer.lr))

# ==================================================================
#
show_history = Show_History()
change_lr = LearningRateScheduler(scheduler)
#tbCallBack = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False,
#                                         embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
nanstop = keras.callbacks.TerminateOnNaN()
reducelearate = keras.callbacks.ReduceLROnPlateau(monitor='lr', factor=0.5, patience=2, min_lr=1e-10)
earlystop = keras.callbacks.EarlyStopping(monitor='lr', min_delta=3, patience=0, verbose=0, mode='min')

if LOAD_FROM_WEIGHTS:
  mbllen.load_weights(ROOT_DIR + '/models/em/' + MODEL_WEIGHTS_FILE)

combined.fit_generator(
        data_loader.load_data(BATCH_SIZE),
        steps_per_epoch=STEPS_PER_EPOCH, # num_samples // batch_size
        epochs=NUM_EPOCHS,
        callbacks=[show_history, change_lr, nanstop, reducelearate])
print('Done!')


Model: "model_38"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_38 (InputLayer)           [(None, 256, 256, 6) 0                                            
__________________________________________________________________________________________________
lambda_72 (Lambda)              (None, 256, 256, 3)  0           input_38[0][0]                   
__________________________________________________________________________________________________
model_36 (Model)                (None, 256, 256, 3)  587727      lambda_72[0][0]                  
__________________________________________________________________________________________________
lambda_73 (Lambda)              (None, 256, 256, 3)  0           input_38[0][0]                   
___________________________________________________________________________________________